In [ ]:
from ultralytics import YOLO

# Load a pretrained model
model = YOLO('yolov8n.pt')

# Train the model
results = model.train(data='data.yaml', epochs=230, imgsz=640, device=0)

In [ ]:
model = YOLO('runs/detect/train/weights/best.pt')

# Validate the model
metrics = model.val()
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

In [ ]:
# Prediction using trained model
model.predict('test_images', save=True, imgsz=640, conf=0.2)

In [ ]:
# Export the model
model.export(format='tflite')

In [ ]:
from ultralytics import YOLO
import cv2
import math 

model = YOLO('runs/detect/train/weights/best.pt')

# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
classNames= ['10 Diamonds', '10 Hearts', '10 Spades', '10 Trefoils', '2 Diamonds', '2 Hearts', '2 Spades', '2 Trefoils', '3 Diamonds', '3 Hearts', '3 Spades', '3 Trefoils', '4 Diamonds', '4 Hearts', '4 Spades', '4 Trefoils', '5 Diamonds', '5 Hearts', '5 Spades', '5 Trefoils', '59', '6 Diamonds', '6 Hearts', '6 Spades', '6 Trefoils', '7 Diamonds', '7 Hearts', '7 Spades', '7 Trefoils', '8 Diamonds', '8 Hearts', '8 Spades', '8 Trefoils', '9 Diamonds', '9 Hearts', '9 Spades', '9 Trefoils', 'A Diamonds', 'A Hearts', 'A Spades', 'A Trefoils', 'J Diamonds', 'J Hearts', 'J Spades', 'J Trefoils', 'K Diamonds', 'K Hearts', 'K Spades', 'K Trefoils', 'Q Diamonds', 'Q Hearts', 'Q Spades', 'Q Trefoils']

while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0]*100))/100
            print("Confidence --->",confidence)

            # class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

    cv2.imshow('Webcam',img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()